In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.functions import array_to_vector
from pyspark.sql import functions as F

In [2]:
vectorized_df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_vectors_result')
vectorized_df.show()

+--------------------+--------------------+
|        cleaned_text|      output_vectors|
+--------------------+--------------------+
|[one, dead, hurt,...|{1, [0.0361029098...|
|[shelf, silver, s...|{1, [-0.445606872...|
|[year, satyam, fr...|{1, [-0.469876835...|
|       [sonic, boom]|{1, [-0.352170139...|
|          [cat, men]|{1, [0.0791339576...|
|[rebuild, holmes,...|{1, [0.0957412049...|
|[be, mahindras, s...|{1, [-0.161969950...|
|   [form, substance]|{1, [-0.278256021...|
|[medical, insuran...|{1, [-0.438288903...|
|[shah, rukh, twee...|{1, [-0.008773446...|
|[trouble, snow, a...|{1, [-0.124195749...|
|[next, bill, gate...|{1, [-0.377679947...|
|[indian, villages...|{1, [-0.317798322...|
|[indian, youth, s...|{1, [-0.178084619...|
|[temple, sao, jos...|{1, [-0.015021715...|
|[govt, ease, educ...|{1, [-0.677946190...|
|[india, beat, tur...|{1, [-0.386804085...|
|[musharraf, meet,...|{1, [-0.375249467...|
|[india, war, mong...|{1, [-0.132766443...|
|[us-pakistan, bic...|{1, [-0.15

In [3]:
vectorized_df.printSchema()

root
 |-- cleaned_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- output_vectors: struct (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)



In [4]:
vectorized_df = vectorized_df.withColumn('output_vectors', array_to_vector('output_vectors.values'))
vectorized_df.show()

+--------------------+--------------------+
|        cleaned_text|      output_vectors|
+--------------------+--------------------+
|[one, dead, hurt,...|[0.03610290982760...|
|[shelf, silver, s...|[-0.4456068724393...|
|[year, satyam, fr...|[-0.4698768357435...|
|       [sonic, boom]|[-0.3521701395511...|
|          [cat, men]|[0.07913395762443...|
|[rebuild, holmes,...|[0.09574120491743...|
|[be, mahindras, s...|[-0.1619699504226...|
|   [form, substance]|[-0.2782560214400...|
|[medical, insuran...|[-0.4382889032363...|
|[shah, rukh, twee...|[-0.0087734460830...|
|[trouble, snow, a...|[-0.1241957495609...|
|[next, bill, gate...|[-0.3776799477636...|
|[indian, villages...|[-0.3177983223771...|
|[indian, youth, s...|[-0.1780846191104...|
|[temple, sao, jos...|[-0.0150217153131...|
|[govt, ease, educ...|[-0.6779461907488...|
|[india, beat, tur...|[-0.3868040855973...|
|[musharraf, meet,...|[-0.3752494673244...|
|[india, war, mong...|[-0.1327664434909...|
|[us-pakistan, bic...|[-0.157507

In [5]:
vectorized_df.count()

40954102

In [6]:
kmeans = KMeans(featuresCol='output_vectors', predictionCol='prediction', k=5, maxIter=10, seed=1)
model = kmeans.fit(vectorized_df)

In [ ]:
model.write().overwrite().save('/common/users/shared/cs543_fall22_group3/models/k_means')

In [8]:
model.clusterCenters()

[array([-0.62283286]),
 array([0.23639457]),
 array([-0.36741422]),
 array([-0.17766151]),
 array([0.01256126])]

In [9]:
transformed = model.transform(vectorized_df)

In [10]:
# 0 -> pop culture
# 1 -> finance
# 2 -> politics
# 3 -> breaking news
# 4 -> sports/entertainment

In [11]:
transformed.filter(transformed.prediction == 3).head(10)

[Row(cleaned_text=['be', 'mahindras', 'sell', 'lemon'], output_vectors=DenseVector([-0.162]), prediction=3),
 Row(cleaned_text=['trouble', 'snow', 'afghanistan'], output_vectors=DenseVector([-0.1242]), prediction=3),
 Row(cleaned_text=['indian', 'youth', 'set', 'australia'], output_vectors=DenseVector([-0.1781]), prediction=3),
 Row(cleaned_text=['india', 'war', 'monger', 'country', 'antony'], output_vectors=DenseVector([-0.1328]), prediction=3),
 Row(cleaned_text=['us-pakistan', 'bicker', 'get', 'ugly'], output_vectors=DenseVector([-0.1575]), prediction=3),
 Row(cleaned_text=['india', 'aim', 'upstage', 'rampage', 'lankans'], output_vectors=DenseVector([-0.1428]), prediction=3),
 Row(cleaned_text=['india', 'war', 'monger', 'country', 'antony'], output_vectors=DenseVector([-0.1328]), prediction=3),
 Row(cleaned_text=['ibf', 'float', 'tender', 'exclusive', 'market', 'agent'], output_vectors=DenseVector([-0.1654]), prediction=3),
 Row(cleaned_text=["'re", 'vigorously', 'follow', 'attack',

In [12]:
# 0 -> 
# 1 -> technology?
# 2 -> politics
# 3 -> finance
# 4 -> sports
# 8 -> health